In [436]:
import pandas as pd 
import numpy as np 
import sklearn

In [437]:
df=pd.read_csv("C:\Customer_Churn_project\customer_churn_model\src\customer_churn_dataset2.csv")

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\vishakha\AppData\Local\Temp\ipykernel_29092\3810052883.py:1: SyntaxWarning: invalid escape sequence '\C'
  df=pd.read_csv("C:\Customer_Churn_project\customer_churn_model\src\customer_churn_dataset2.csv")


In [438]:
df.head(5)

,customer_id,age,location,signup_date,subscription_type,last_payment_date,payment_delay_days,failed_payments_count,avg_weekly_usage_hours,days_since_last_login,courses_completed,tickets_raised,last_ticket_days
0,1,58,Chicago,704,Yearly,4,6,2,2.67,676,9,0,NaN
1,2,18,Chicago,410,Yearly,161,11,2,9.40,252,3,0,NaN
2,3,30,Austin,104,Yearly,47,5,1,7.24,68,9,4,128.0
3,4,47,Los Angeles,566,Monthly,495,15,4,14.30,475,4,2,295.0
4,5,46,New York,126,Yearly,115,14,3,15.46,95,3,5,290.0


In [439]:
df.isnull().sum()

customer_id                 0
age                         0
location                    0
signup_date                 0
subscription_type           0
last_payment_date           0
payment_delay_days          0
failed_payments_count       0
avg_weekly_usage_hours      0
days_since_last_login       0
courses_completed           0
tickets_raised              0
last_ticket_days          508
dtype: int64

In [440]:

df['signup_date'] = pd.to_datetime(df['signup_date'])
df['last_payment_date'] = pd.to_datetime(df['last_payment_date'])


today = pd.Timestamp.today().normalize()

df['days_since_signup'] = (today - df['signup_date']).dt.days
df['days_since_last_payment'] = (today - df['last_payment_date']).dt.days



In [441]:
df = df.drop(columns=['signup_date', 'last_payment_date'])


In [442]:
df['last_ticket_days'] = df['last_ticket_days'].fillna(0)

In [443]:
df["payment_delay_flag"] = np.where(df["payment_delay_days"] > 7, 1, 0)
df["low_usage_flag"] = np.where(df["avg_weekly_usage_hours"] < 3, 1, 0)
df["support_intensity"] = df["tickets_raised"] / (df["courses_completed"] + 1)


In [444]:
df['engagement_rate'] = df['avg_weekly_usage_hours'] / (df['days_since_signup'] + 1)

df['activity_drop_flag'] = (df['days_since_last_login'] > 14).astype(int)


df['payment_stress'] = df['payment_delay_days'] * (df['failed_payments_count'] + 1)


df['support_pressure'] = df['tickets_raised'] / (df['days_since_signup'] + 1)


In [445]:
def handle_outliers_iqr(df,column):
    Q1=df[column].quantile(0.25)
    Q3=df[column].quantile(0.75)
    IQR=Q3-Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    #capping not removing the data
    df[column] = df[column].clip(lower=Q1 - 1.5 * IQR, upper=Q3 + 1.5 * IQR)
    print(f"{column} → Lower: {lower_bound}, Upper: {upper_bound}")

    return df

numeric_columns=df.select_dtypes(include=['number']).columns
for col in numeric_columns:
    df = handle_outliers_iqr(df, col)





customer_id → Lower: -1498.5, Upper: 4499.5
age → Lower: -5.0, Upper: 83.0
payment_delay_days → Lower: -8.0, Upper: 24.0
failed_payments_count → Lower: -2.0, Upper: 6.0
avg_weekly_usage_hours → Lower: -9.314999999999996, Upper: 30.184999999999995
days_since_last_login → Lower: -312.5, Upper: 651.5
courses_completed → Lower: -7.0, Upper: 17.0
tickets_raised → Lower: -3.5, Upper: 8.5
last_ticket_days → Lower: -296.5, Upper: 595.5
days_since_signup → Lower: 20485.0, Upper: 20485.0
days_since_last_payment → Lower: 20485.0, Upper: 20485.0
payment_delay_flag → Lower: -1.5, Upper: 2.5
low_usage_flag → Lower: 0.0, Upper: 0.0
support_intensity → Lower: -0.8333333333333334, Upper: 1.8333333333333335
engagement_rate → Lower: -0.00045470077125842035, Upper: 0.0014734452797032119
activity_drop_flag → Lower: 1.0, Upper: 1.0
payment_stress → Lower: -32.5, Upper: 75.5
support_pressure → Lower: -0.00017084838426242312, Upper: 0.00041491750463731327


In [446]:
df.dtypes

customer_id                  int64
age                          int64
location                       str
subscription_type              str
payment_delay_days           int64
failed_payments_count        int64
avg_weekly_usage_hours     float64
days_since_last_login      float64
courses_completed            int64
tickets_raised               int64
last_ticket_days           float64
days_since_signup            int64
days_since_last_payment      int64
payment_delay_flag           int64
low_usage_flag               int64
support_intensity          float64
engagement_rate            float64
activity_drop_flag           int64
payment_stress               int64
support_pressure           float64
dtype: object

In [447]:
df.head(5)

,customer_id,age,location,subscription_type,payment_delay_days,failed_payments_count,avg_weekly_usage_hours,days_since_last_login,courses_completed,tickets_raised,last_ticket_days,days_since_signup,days_since_last_payment,payment_delay_flag,low_usage_flag,support_intensity,engagement_rate,activity_drop_flag,payment_stress,support_pressure
0,1,58,Chicago,Yearly,6,2,2.67,651.5,9,0,0.0,20485,20485,0,0,0.00,0.000130,1,18,0.000000
1,2,18,Chicago,Yearly,11,2,9.40,252.0,3,0,0.0,20485,20485,1,0,0.00,0.000459,1,33,0.000000
2,3,30,Austin,Yearly,5,1,7.24,68.0,9,4,128.0,20485,20485,0,0,0.40,0.000353,1,10,0.000195
3,4,47,Los Angeles,Monthly,15,4,14.30,475.0,4,2,295.0,20485,20485,1,0,0.40,0.000698,1,75,0.000098
4,5,46,New York,Yearly,14,3,15.46,95.0,3,5,290.0,20485,20485,1,0,1.25,0.000755,1,56,0.000244


In [448]:
df['location'].value_counts()

location
Chicago              750
New York             747
Los Angeles          745
Seattle              155
Austin               150
                    ... 
North Ashleyville      1
South Lisaport         1
Piercehaven            1
Wilsonside             1
Petersonmouth          1
Name: count, Length: 312, dtype: int64

In [449]:
df=df.drop(columns='location')

In [450]:
df.head(5)

,customer_id,age,subscription_type,payment_delay_days,failed_payments_count,avg_weekly_usage_hours,days_since_last_login,courses_completed,tickets_raised,last_ticket_days,days_since_signup,days_since_last_payment,payment_delay_flag,low_usage_flag,support_intensity,engagement_rate,activity_drop_flag,payment_stress,support_pressure
0,1,58,Yearly,6,2,2.67,651.5,9,0,0.0,20485,20485,0,0,0.00,0.000130,1,18,0.000000
1,2,18,Yearly,11,2,9.40,252.0,3,0,0.0,20485,20485,1,0,0.00,0.000459,1,33,0.000000
2,3,30,Yearly,5,1,7.24,68.0,9,4,128.0,20485,20485,0,0,0.40,0.000353,1,10,0.000195
3,4,47,Monthly,15,4,14.30,475.0,4,2,295.0,20485,20485,1,0,0.40,0.000698,1,75,0.000098
4,5,46,Yearly,14,3,15.46,95.0,3,5,290.0,20485,20485,1,0,1.25,0.000755,1,56,0.000244


In [451]:
df.groupby('subscription_type').size()

subscription_type
Monthly    1523
Yearly     1477
dtype: int64

In [452]:
df["churn"] = np.where(
    ((df["days_since_last_login"] > 30) & (df["failed_payments_count"] >= 1)) 
    | (df["avg_weekly_usage_hours"] < 2),
    1, 0
)


In [453]:

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
import joblib





In [454]:
df.columns

Index(['customer_id', 'age', 'subscription_type', 'payment_delay_days',
       'failed_payments_count', 'avg_weekly_usage_hours',
       'days_since_last_login', 'courses_completed', 'tickets_raised',
       'last_ticket_days', 'days_since_signup', 'days_since_last_payment',
       'payment_delay_flag', 'low_usage_flag', 'support_intensity',
       'engagement_rate', 'activity_drop_flag', 'payment_stress',
       'support_pressure', 'churn'],
      dtype='str')

In [455]:
df['churn'].value_counts(normalize=True)


churn
1    0.694333
0    0.305667
Name: proportion, dtype: float64

In [456]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OrdinalEncoder(categories=[["Monthly", "Yearly"]]),["subscription_type"])
    ],
    remainder="passthrough"  # keep numeric columns as-is
)

pipeline = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("rfc", RandomForestClassifier())
])

In [463]:
X = df.drop(columns=["churn","days_since_last_login","failed_payments_count","avg_weekly_usage_hours"])
y = df["churn"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [466]:
clean_list = X_test.iloc[[0]].values.tolist()[0]
print(clean_list)

[1802, 47, 'Yearly', 5, 8, 0, 0.0, 20485, 20485, 0, 0, 0.0, 0.0003363272478765986, 1, 5, 0.0]


In [458]:
pipeline.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('rfc', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains s

In [459]:
pred = pipeline.predict(X_test)

In [460]:
print("Accuracy:", accuracy_score(y_test, pred))

Accuracy: 0.8483333333333334
